# Project: MongoDB

The purpose of this assignment is to:
- Familarize with JSON document syntax
- Understand basic MongoDB CRUD operations
- Understand MongoDB data pipelines to run aggregate queries

This dataset has 3 collections: Employee, Workplace and Address.

## Setup Notes

We will be using MongoDB Community Edition.  The MongoDB database <b>MUST</b> be installed and running locally before continuing with this notebook.  We will need to install two packages using the Anaconda package manager:

1. [Mongodb](https://www.mongodb.com/) - this package contains the mongodb database 
2. [PyMongo](http://api.mongodb.com/python/current/) - this packages contains the python driver that will allow us to communicate with the mongodb database.

### Install

1. Open a command line terminal and execute the following commad to install mongodb.
```console
conda install -c anaconda mongodb
```
2. Open a command line terminal and execute the following commad to install pymongo packge.
```console
conda install -c anaconda pymongo
```

#### Run Mongodb in Windows
1. MongoDB requires a data directory to store all data. MongoDB’s default data directory path is \data\db. Create this folder using the following commands from a Command Prompt:
```console
md \data\db
```

2. To start MongoDB, run mongod.exe. For example, from the Command Prompt:
```console
"C:\Program Files\MongoDB\Server\3.2\bin\mongod.exe"
```

#### Run Mongodb in Mac
1. MongoDB requires a data directory to store all data. MongoDB’s default data directory path is /data/db. Create this folder using the following commands from a Command Prompt.  Note that we run the command as a super user using the "sudo" command:
```console
sudo mkdir /data/db/
```

2. To start MongoDB, run mongod.exe. For example, from the Command Promp.  Note that we run the command as a super user using the "sudo" command:
```console
sudo mongod --config /usr/local/etc/mongod.conf
```

# Student: Matthew Kwok

In [1]:
#required imports
import os
import json
import datetime
import pymongo
import pprint
import pandas as pd
import numpy as np
from pymongo import MongoClient
print('Mongo version', pymongo.__version__)

Mongo version 3.11.0


We first need to connect to our locally running MongoDB database (<b>Make sure your database is running on your machine</b>). We will use the MongoClient to connect to a local 'test' database that is running on port 27017 (this is the default port).

In [2]:
client = MongoClient('localhost', 27017)
db = client.assignment1

After installing necessary modules proceed to import the data into your database.

In [3]:
# Let's delete any existing collections in our database
db.workplace.drop()
db.address.drop()
db.employee.drop()

# Import our files into our three collections
with open('Data/Employee.json') as f:
    db.employee.insert_many(json.load(f))
with open('Data/Workplace.json') as f:
    db.workplace.insert_many(json.load(f))
with open('Data/Address.json') as f:
    db.address.insert_many(json.load(f))

### Part 1

The address collection contains employee from different ages and interests.  Perform a simple query to list all employees that are less than or equal to 50 and like Cooking.

In [4]:
pprint.pprint(client.assignment1.employee.find_one())

{'_id': '9f39da36-82cc-4353-ab90-d616105fa7c1',
 'address_id': 'b6c0b50a-d0e3-43bf-a2a4-8d4674c2a7e8',
 'age': 40,
 'email': 'ih@ri.ro',
 'firstname': 'Emilie',
 'interests': ['Bowling', 'Cooking', 'Golf', 'Swimming'],
 'lastname': 'Woods',
 'workplace_id': 'a32bf18d-e0e5-48f2-a851-aa49c80f9460'}


In [5]:
# find employees 
cursor = db.employee.find(
    {
        'age': {'$lte': 50},      # less than or equal to 50 yrs old
        'interests': 'Cooking'    # interested in cooking
    }, 
    {'firstname', 'lastname', 'age', 'interests'}
)

# display cursor results in dataframe
pd.DataFrame(list(cursor)) 

,_id,age,firstname,interests,lastname
0,9f39da36-82cc-4353-ab90-d616105fa7c1,40,Emilie,"[Bowling, Cooking, Golf, Swimming]",Woods
1,af27265e-6639-49f2-991e-193275a4111a,18,Thomas,"[Cooking, Cricket, Tennis, Swimming, Fishing]",Patterson
2,00289d48-bad8-4b73-a359-a1a1f05c96e2,22,Sophia,"[Hiking, Soccer, Bowling, Rubgy, Cooking, Danc...",Flores
3,da76e52b-b3db-4fc0-b0d6-435d1aed0cd9,25,Ollie,"[Cooking, Bowling, Dancing]",Barnett
4,51643cd6-49bb-45d5-bd6e-717c62bb2869,27,James,"[Rubgy, Tennis, Cricket, Cooking]",Wilkins
5,f073a705-6546-4375-adb5-b224871776ef,25,Aaron,[Cooking],Carr
6,457ef68c-9651-4925-bca0-15e246661d19,34,Alta,"[Cricket, Cycling, Rubgy, Golf, Cooking, Dancing]",Sharp
7,840184a3-4c4d-4b15-8813-30fca6e7827b,36,Delia,"[Cricket, Cooking, Hiking, Dancing, Tennis]",Douglas
8,6157dc3b-ee2d-463a-b62b-1cd4eed7d575,48,Dominic,"[Cycling, Dancing, Cooking]",Wade
9,cc3e389d-be0d-467a-ba70-7c84f6504911,36,Myrtle,"[Cooking, Cycling, Hiking, Rubgy, Bowling, Dan...",Little


### Part 3

Insert a new Employee with the following properties:

* First Name: Jake 
* Last Name: Sample
* Email: jakesample@email.com
* Age: 26
* Interest: Biking, Hiking

Also, this employee works for 'Union Planters Corp' and lives at '573 Wojhas Square, Victoria'.
Verify that the insert succeeded and display the generated employees _id attribute.

In [11]:
db.employee.insert_one(
    {
        'firstname': 'Jake',
        'lastname': 'Sample',
        'age': 40,
        'email': 'jakesample@email.com',
        'interests': ['Biking', 'Hiking'],
        'address_id': '91b5b7b3-2309-4e8a-8247-cd66d626ef0c',
        'workplace_id': '5345fcb9-6297-4b9f-aa15-cbee8460f28f'
    }
)

# show the last 5 entries in employee document in descending order
query_result = db.employee.find()\
              .sort(
                    '$natural',  # sort based on the order the document was inserted
                     pymongo.DESCENDING) \
              .limit(5) # only look at the last 5 inserted documents

pd.DataFrame(list(query_result))

,_id,address_id,age,email,firstname,interests,lastname,workplace_id
0,6025e2bede4068417db714a3,91b5b7b3-2309-4e8a-8247-cd66d626ef0c,40,jakesample@email.com,Jake,"[Biking, Hiking]",Sample,5345fcb9-6297-4b9f-aa15-cbee8460f28f
1,6025e218de4068417db714a2,91b5b7b3-2309-4e8a-8247-cd66d626ef0c,40,jakesample@email.com,Jake,"[Biking, Hiking]",Sample,5345fcb9-6297-4b9f-aa15-cbee8460f28f
2,663e70d1-1b08-4e40-a880-befe2bec42d3,824fea9d-d0e6-4c53-ad9d-5bf626a948f3,52,bibtar@ru.gb,Edward,"[Hiking, Soccer, Dancing]",Hawkins,2b87eb84-e5b4-4f2c-9e13-dc3ba20a7f7f
3,2bfc0479-cb15-46e1-94fa-801404791b21,77ce0c50-afd6-47e8-86d7-b1b372e8deb2,37,usope@tega.bh,Peter,"[Cooking, Swimming, Cricket]",Jimenez,cb795afb-8dc3-482f-b3a0-14229a280afa
4,b2a2ae86-f4f6-4da5-ade6-ee1cd135daf3,ef5a17a2-6f11-49b8-a0fd-e0bddc56f185,49,ah@jopah.uz,Gavin,"[Rubgy, Cooking, Bowling]",Conner,50275ad1-8140-4e79-8818-21793e3eb0a3


### Part 3

Delete all employees that work for 'Great Plains Energy Inc' and are greater than 46 years old and likes 'Tennis'. Once you delete the employees verify the number of employees deleted.

In [7]:
# 'Great Plains Energy Inc.' workplace_id = 'a32bf18d-e0e5-48f2-a851-aa49c80f9460'

# finding the employees that fit the conditions
locate = db.employee.find(
    {
        'age': {'$gt': 46},
        'interests': 'Tennis',
        'workplace_id': 'a32bf18d-e0e5-48f2-a851-aa49c80f9460'
    },
    {'firstname', 'lastname', 'age', 'interests', 'workplace_id'}
)

# delete the employees
cursor_2 = db.employee.delete_many(
    {
        'age': {'$gt': 46},
        'interests': 'Tennis',
        'workplace_id': 'a32bf18d-e0e5-48f2-a851-aa49c80f9460'
    }
)

print('Delete Acknowledged:' + str(cursor_2.acknowledged))             # checks if delete was carried out
print('Number of employees deleted: ' + str(cursor_2.deleted_count))

Delete Acknowledged:True
Number of employees deleted: 4


### Part 4
Add a new field called 'industry' to all employees that work for 'Health Net Inc.'.

In [15]:
# 'Health Net Inc.' workplace_id = 'a222385c-342c-43ea-adbc-9b487a2ee2be'

db.employee.update_many(
    {'workplace_id': 'a222385c-342c-43ea-adbc-9b487a2ee2be'},   # condition
    {'$set': {'industry': 'NAN'}}     
)

pd.DataFrame(list(db.employee.find(
    {'workplace_id': 'a222385c-342c-43ea-adbc-9b487a2ee2be'}
)))

,_id,address_id,age,email,firstname,industry,interests,lastname,workplace_id
0,ad4fd8bd-b732-4388-8469-5395b497df63,91b32548-65b6-4fd9-91a1-2d86b04b8eea,42,tasnimnag@tefifdum.bo,Marc,NAN,"[Golf, Cricket, Rubgy, Hiking, Fishing, Dancing]",Castro,a222385c-342c-43ea-adbc-9b487a2ee2be
1,6222b4f5-6fd3-4c72-b058-710fb568b869,59516270-19f4-47e8-95ae-3e83fa5f230c,46,ruupeke@oh.om,Lucinda,NAN,"[Golf, Rubgy, Tennis]",Gomez,a222385c-342c-43ea-adbc-9b487a2ee2be
2,ca189ae9-d04b-4ae0-9b4a-b725196c31d2,16ff1778-b687-42ef-aace-36cedc05f521,37,vom@calfim.se,Emilie,NAN,[Hiking],Robertson,a222385c-342c-43ea-adbc-9b487a2ee2be
3,ed499274-f17f-4a4c-8327-7e740e6dd9c1,01185f9b-b00f-4f2d-9878-0c989fe00fbc,36,newi@to.in,Josephine,NAN,"[Tennis, Bowling, Cricket]",Mills,a222385c-342c-43ea-adbc-9b487a2ee2be
4,6a3bd505-56e6-46ec-bef1-2eef1fd3643c,a8f07070-cfb0-483e-a07a-b2adf9e3d4b0,24,disbuivi@efu.id,Vernon,NAN,[Soccer],Edwards,a222385c-342c-43ea-adbc-9b487a2ee2be
5,5b8bb404-5741-46a9-84f2-81248ffd2840,fc2b6b58-8392-4f3d-b1c8-07f4df87dc24,43,kazvasi@nojrek.aq,Glen,NAN,"[Golf, Hiking]",Turner,a222385c-342c-43ea-adbc-9b487a2ee2be
6,c7cf8650-f391-4f2b-bbb5-60225729e998,44cc24b9-1d60-45cf-be0d-a225f35d52ab,49,himosewej@repan.bw,Jeremiah,NAN,"[Cricket, Fishing]",Payne,a222385c-342c-43ea-adbc-9b487a2ee2be
7,9ebcf9fc-e193-4ab7-9c38-10c2d6bf6036,d34f9d8c-c1a0-4653-8a44-d2fe65714956,76,zut@zoksa.kw,Alan,NAN,"[Bowling, Cooking, Rubgy, Fishing, Golf, Cricket]",Berry,a222385c-342c-43ea-adbc-9b487a2ee2be
8,fd677b4b-6eef-4777-bc3a-25d9a86134fa,98fa15f0-5b73-4fb9-a5a4-1332e0f5b58b,37,pelohno@hotivgu.hn,Mattie,NAN,"[Cycling, Fishing, Soccer, Rubgy, Hiking]",Fisher,a222385c-342c-43ea-adbc-9b487a2ee2be
9,cb369c19-4bbc-449f-bff6-b198439d0d7b,d5b7473d-c82b-4956-bce3-80497ce6c015,43,oka@lilozfaz.gn,Adele,NAN,"[Swimming, Hiking, Tennis]",Benson,a222385c-342c-43ea-adbc-9b487a2ee2be


### Part 5

Create an aggregate query to count the number of employees for each company and sort the output from largest employee count to lowest employee count.

<table>
    <tr><th></th><th>_id</th><th>count</th></tr>
    <tr><td>0</td><td>[Equity Residential Properties Trust]</td><td>19</td></tr>
    <tr><td>...</td><td>...</td><td>...</td></tr>
    <tr><td>7</td><td>[Bell Microproducts Inc.]</td><td>6</td></tr>
    <tr><td>8</td><td>[Kemet Corp.]</td><td>1</td></tr>
</table>

In [9]:
pipeline = [
    {'$lookup':
     {
         'from': 'workplace',   # in workplace doc find...
         'localField': 'workplace_id',  # get workplace_id from the document being analyzed (the local one)
         'foreignField': '_id',        # match the local document id to id of the workplace in workplace document
         'as': 'workplace'
     }     
    },
    {'$unwind': '$workplace'},
    {'$group': 
     {
         '_id': '$workplace.name', 
         'Number of Employees': {"$sum": 1}
     }
    },
    {'$sort': {'Number of Employees': -1}}   # sort by decending order
]

query_result = db.employee.aggregate(pipeline)

pd.DataFrame(list(query_result))

,Number of Employees,_id
0,15,Hilton Solutions
1,14,Health Net Inc.
2,13,Aetna Inc.
3,11,Bell Microproducts Inc.
4,10,Equity Office Properties Trust
5,10,Union Planters Corp
6,7,Equity Residential Properties Trust
7,6,Xcel Bear Inc
8,6,Kemet Corp.
9,5,Great Plains Energy Inc.
